# Notebook 1: Introduction with Shallow Water - Williamson 2 Test Case

This notebook provides an introduction to Gusto. We will go through the example of the Williamson 2 test case to demonstrate how to set up the problem and run it. 

The Williamson 2 test case solves the shallow water equations. This corresponds to the following set of momentum and continuity equations, for a velocity $\textbf{u}$ and free surface, $\eta$:
$$\textbf{u}_t + f \textbf{u}^{\perp} + g \nabla \eta + (\textbf{u} \cdot \nabla) \textbf{u} = 0$$
$$\eta_t + H(\nabla \cdot \textbf{u}) + \nabla [\textbf{u} (\eta - b)] = 0$$

The parameters we need to specify are the Coriolis force, $f$, gravitational constant, $g$, and mean depth, $H$.
$\newline$
We can neglect boundary conditions, due to solving on a spherical domain. 

We begin by importing the required libraries and functions from Firedrake and Gusto. This step is required anytime you are using Gusto.

In [1]:
from gusto import *
from firedrake import IcosahedralSphereMesh, SpatialCoordinate, as_vector
from math import pi
import sys

firedrake:WARNING OMP_NUM_THREADS is not set or is set to a value greater than 1, we suggest setting OMP_NUM_THREADS=1 to improve performance


We now define the time-step size and the duration of the simulation. We will also decide how frequently we want an output to be able to visualise in Paraview, using dumpfreq.

In [3]:
dt = 4000.
day = 24.*60.*60.
tmax = 1*day

ndumps = 5
dumpfreq = int(tmax / (ndumps*dt))

As the spherical domain we are solving over is the Earth, we specify a relevant radius and height to be able to construct the mesh.

In [4]:
# setup shallow water parameters
R = 6371220.
H = 5960.

# setup input that doesn't change with ref level or dt
parameters = ShallowWaterParameters(H=H)

Next, we we will construct an Icosahedral Sphere Mesh. We specify a refinement level, which specifies how fine the spatial discretisation of the mesh is.

In [5]:
#Set up the mesh and choose the refinement level
ref_level = 3  # number of horizontal cells = 20*(4^refinements)

mesh = IcosahedralSphereMesh(radius=R,
                             refinement_level=ref_level, degree=3)
x = SpatialCoordinate(mesh)
mesh.init_cell_orientations(x)

We need to specify an output directory name before running the code. The naming convention we will use records the test case, refinement level and time-step size. \
To prevent losing hard-earned simulation data, Gusto will not enable overwriting an existing file. Hence, if one wishes to re-run a simulation with the same output filename, the existing results file needs to be deleted first. 

In [6]:
output = OutputParameters(dirname="sw_W2_ref%s_dt%s" % (ref_level, dt),
                          dumpfreq=dumpfreq,
                          steady_state_error_fields=['u', 'D'],
                          log_level='INFO')

We can specify which diagnostics we wish to record over a simulation. The list of avaliable diagnostics can be found in the gusto source code: https://github.com/firedrakeproject/gusto/blob/main/gusto/diagnostics.py \
We pass these diagnostics into the State function. This will initialise all the necessary components of the test case. Gusto will update this state after each iteration of the chosen time-stepper.

In [9]:
diagnostic_fields = [RelativeVorticity(), PotentialVorticity(),
                         ShallowWaterKineticEnergy(),
                         ShallowWaterPotentialEnergy(),
                         ShallowWaterPotentialEnstrophy(),
                        CourantNumber()]

state = State(mesh,
              dt=dt,
              output=output,
              parameters=parameters,
              diagnostic_fields=diagnostic_fields)

gusto:INFO Physical parameters that take non-default values:
gusto:INFO H: 5960.0


A spatially varying Coriolis force is defined over the sphere. We then can set-up our form of the shallow water equations by passing the state and Coriolis parameter into the ShallowWaterEquations() function. We additionally specify the function space (BDM) and degree (1) that we want to use with the Compatiable Finite Element method. 

In [10]:
#Create a spatially varying function for the Coriolis force:
Omega = parameters.Omega
x = SpatialCoordinate(mesh)
fexpr = 2*Omega*x[2]/R
eqns = ShallowWaterEquations(state, "BDM", 1, fexpr=fexpr)

We are now ready to specify the initial conditions.
Due to our choice of function spaces for the velocity and depth, the intialisations of each variable use projection 
and interpolation operations respectively. Williamson 2 specifies initial conditions for a 'Global Steady State Non-linear Zonal Geostrophic Zone' (do I need this detail?). 
$$ \textbf{u}_0 = \frac{u_{max}}{R} [-y,x,0] $$
$$ h_0 = H - \frac{\Omega u_{max} z^2}{g R}  $$
We only require a two-dimensional velocity, so the vertical component is specified to be zero. (Jemma/Tom question: Why is $R/R^2$ used in the code)???

In [11]:
u0 = state.fields("u")
D0 = state.fields("D")
u_max = 2*pi*R/(12*day)  # Maximum amplitude of the zonal wind (m/s)
uexpr = as_vector([-u_max*x[1]/R, u_max*x[0]/R, 0.0])
g = parameters.g
Dexpr = H - ((R * Omega * u_max)*(x[2]*x[2]/(R*R)))/g
u0.project(uexpr)
D0.interpolate(Dexpr)

Coefficient(WithGeometry(IndexedProxyFunctionSpace(<firedrake.mesh.MeshTopology object at 0x7fbccb28cdf0>, FiniteElement('Discontinuous Lagrange', triangle, 1, variant='equispaced'), name='DG1', index=1, component=None), Mesh(VectorElement(FiniteElement('Lagrange', Cell('triangle', 3), 3), dim=3), 4)), 26)

Now we will choose a time-stepper. We will demostrate the 'Crank-Nicolson' approach here; this allows for different time-steppers to be used for evolving the velocity and depth fields. We choose to use an Implicit Midpoint method for the velocity and an explicit strong stability preserving RK3 method for the depth. A full list of avaliable time stepping methods can be found at: https://github.com/firedrakeproject/gusto/blob/main/gusto/time_discretisation.py

In [11]:
#Now, construct the time-stepper. We will firstly use a semi-implicit (?) approach.
#We will neglect any transport schemes for now.
transport_schemes = [ImplicitMidpoint(state, "u"),
                          SSPRK3(state, "D", subcycles=2)]
stepper = CrankNicolson(state, eqns, transport_schemes)

We are ready to run our simulation! We simply instruct the time-stepper to run for the specified duration.

In [12]:
#Run the time-stepper and generate the output
stepper.run(t=0, tmax=tmax)

/home/timandrews/firedrake/src/firedrake/firedrake/checkpointing.py:89: DeprecationWarning: DumbCheckpoint class will be deprecated after 01/01/2023; use CheckpointFile class instead.
  warnings.warn("DumbCheckpoint class will be deprecated after 01/01/2023; use CheckpointFile class instead.",
gusto:INFO at start of timestep, t=0, dt=4000.0
gusto:INFO at start of timestep, t=4000.0, dt=4000.0
gusto:INFO at start of timestep, t=8000.0, dt=4000.0
gusto:INFO at start of timestep, t=12000.0, dt=4000.0
gusto:INFO at start of timestep, t=16000.0, dt=4000.0
gusto:INFO at start of timestep, t=20000.0, dt=4000.0
gusto:INFO at start of timestep, t=24000.0, dt=4000.0
gusto:INFO at start of timestep, t=28000.0, dt=4000.0
gusto:INFO at start of timestep, t=32000.0, dt=4000.0
gusto:INFO at start of timestep, t=36000.0, dt=4000.0
gusto:INFO at start of timestep, t=40000.0, dt=4000.0
gusto:INFO at start of timestep, t=44000.0, dt=4000.0
gusto:INFO at start of timestep, t=48000.0, dt=4000.0
gusto:INFO 

After the simulation is completed, you should see that there is now a 'results' directory, with another directory holding the output. The output at the specified times is saved in a .vtu format that can be viewed using Paraview. There are five files, corresponding to the specified dump times (field_output1.vtu, field_output2.vtu, etc.), as well as one for the initial conditions (field_output0.vtu). Documentation on how to use Paraview can be found here (INSERT). \
Here is what you should see from your Paraview output (TO INSERT FIGURE)

Congratulations, you have now successfully run a Gusto script!